In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/Colab Notebooks')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%matplotlib inline
from copy import deepcopy
from matplotlib import pyplot as plt

import imp
try:
    imp.find_module('jupyterplot')
    from jupyterplot import ProgressPlot
except ImportError:
    !pip install jupyterplot
    from jupyterplot import ProgressPlot

import torch
from torch import nn
from torch.utils.data import DataLoader

import torchvision
from torchvision import datasets as D
from torchvision import transforms as T

from utils import train_step, test_step
from utils import simulate_scheduler
from utils import BaselineModel
from utils import get_cifar10_dataset, make_dataloader

# Learning rate를 실시간으로 조정

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
num_epochs = 30
batch_size = 64
learning_rate = 0.01
momentum = 0.9
phases = ['train', 'test']

In [ ]:
cifar10_dataset = get_cifar10_dataset()
loader = make_dataloader(cifar10_dataset, batch_size)

In [ ]:
base_cnn = BaselineModel()
nets = [
    BaselineModel().to(device)
    for _ in range(2)
]
_ = [net.load_state_dict(deepcopy(base_cnn.state_dict())) for net in nets]

optimizers = [
    torch.optim.SGD(net.parameters(), learning_rate, 0.9)
    for net in nets
]
criterion = nn.CrossEntropyLoss()

In [ ]:
gamma = 0.9
lrs = simulate_scheduler(gamma, num_epochs)
plt.plot(lrs)
plt.xlabel('epoch')
plt.ylabel('learning rate')
plt.show()

In [ ]:
optimizers = [
    torch.optim.SGD(net.parameters(), learning_rate, 0.85)
    for net in nets
]
sched = torch.optim.lr_scheduler.ExponentialLR(optimizers[1], gamma)
conditions = ['without_scheduler', 'with_scheduler']
pp = ProgressPlot(
    plot_names=phases,
    line_names=conditions,
    x_lim=[0, num_epochs*len(loader['train'])],
    x_label='Iteration',
    y_lim=[[0, 3], [50, 100]]
)

accs = [0, 0]
for epoch in range(num_epochs):
    for inputs, target in loader['train']:
        losses = [
            train_step(net, inputs, target, optimizer, criterion, device)
            for net, optimizer in zip(nets, optimizers)
        ]
        pp.update([losses, accs])
    
    corrects = [0, 0]
    for inputs, target in loader['test']:
        outputs = [
            test_step(net, inputs, target, device=device)[0]
            for net in nets
        ]
        corrects = [
            (correct + (output.argmax(1).cpu() == target).sum()).item()
            for correct, output in zip(corrects, outputs)
        ]
        
    accs = [
        correct / len(cifar10_dataset['test']) * 100
        for correct in corrects
    ]
    
    print(f'Epoch: {epoch+1} accuracy ', end='')
    lrs = [learning_rate, sched.get_lr()[0]]
    for cond, acc, lr in zip(conditions, accs, lrs):
        print(f'{cond}: lr {lr:.5f}-{acc:.2f}', end=' ')
    print()
    sched.step()
pp.finalize()